In [6]:
# To destroy.
import digitalocean
import os
import datetime
from tqdm import tqdm
project_to_destroy = "W"
to_destroy = []
manager = digitalocean.Manager( token = os.getenv( 'MARIUS_DOTOKEN' ) ) 
my_droplets = manager.get_all_droplets()
for droplet in my_droplets:
    if len(droplet.tags) > 0 and droplet.tags[0] == project_to_destroy:
        to_destroy.append(droplet)
pbar = tqdm(range(0, len(to_destroy)), "Destroying Droplets")
for droplet in to_destroy:
    pbar.set_description("Destroying %s" % droplet.name)
    droplet.destroy()
    pbar.update(1)


Destroying B48: 100%|██████████| 49/49 [24:28<00:00, 29.97s/it]


In [7]:
# To create.
import digitalocean
import os
from tqdm import tqdm
manager = digitalocean.Manager( token = os.getenv( 'MARIUS_DOTOKEN' ) ) 
project = "R"
slug = 's-8vcpu-16gb'
region = 'nyc3'
n_to_create = 300
keys = manager.get_all_sshkeys()
projects = manager.get_all_projects()
pbar = tqdm(range(0, n_to_create), "Creating Droplets")
for i in pbar:
    name = "{}{}".format(project, i)
    pbar.set_description("Creating %s" % name)
    droplet = digitalocean.Droplet(token = os.getenv( 'MARIUS_DOTOKEN' ), name=name, region=region, image='ubuntu-20-04-x64', size_slug=slug, backups=True, tags = [project], ssh_keys = keys)
    droplet.create()
    for p in projects:
        if p.name == project:
            p.assign_resource(["do:droplet:{}".format(droplet.id)])
            break

Creating R128:  43%|████▎     | 128/300 [02:50<03:14,  1.13s/it]

In [ ]:
# Create wallets
import bittensor
from tqdm import tqdm
project = "R"
manager = digitalocean.Manager( token = os.getenv( 'MARIUS_DOTOKEN' ) ) 
my_droplets = manager.get_all_droplets()
for droplet in my_droplets:
    wallet = bittensor.wallet(name = project, hotkey=droplet.name )
    if not wallet.hotkey_file.exists_on_device():
        wallet.create_new_hotkey()




In [2]:
# Append droplets to config
from hud import HUD
project = "X"
neurons = HUD.load_from_digital_ocean( tag = project )
HUD.write_to_config( neurons )
neurons = HUD.load_from_config( project = project )
print( neurons )


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /var/folders/t2/n8qztvk54892_y4rgslzz_wm0000gn/T/ipykernel_61699/4278052941.py:6 in       │
│ <module>                                                                                  │
│                                                                                           │
│ [Errno 2] No such file or directory:                                                      │
│ '/var/folders/t2/n8qztvk54892_y4rgslzz_wm0000gn/T/ipykernel_61699/4278052941.py'          │
│ /Users/thebes/Work/HUD/hud.py:539 in load_from_config                                     │
│                                                                                           │
│   536 │   def load_from_config( project ) -> 'HUD[Neuron]':                               │
│   537 │   │   with open( 'configs/{}.yaml'.format( project ), "r") as config_file:        │
│   538 │   │   │   config = yaml.safe_load(config_file)                                    │
│ ❱ 539 │   │   config = bittensor.Config.fromDict(config)[project]                         │
│   540 │   │   neurons = []                                                                │
│   541 │   │   for hotkey in config:                                                       │
│   542 │   │   │   name = "{}-{}".format( project, hotkey )                                │
│                                                                                           │
│ ╭───────────────────────────────────── locals ──────────────────────────────────────╮     │
│ │      config = {                                                                   │     │
│ │               │   'x': {                                                          │     │
│ │               │   │   'W0': {                                                     │     │
│ │               │   │   │   'ip_address': '143.198.18.159',                         │     │
│ │               │   │   │   'sshkey': '/Users/thebes/.ssh/marius'                   │     │
│ │               │   │   },                                                          │     │
│ │               │   │   'W1': {                                                     │     │
│ │               │   │   │   'ip_address': '159.65.248.149',                         │     │
│ │               │   │   │   'sshkey': '/Users/thebes/.ssh/marius'                   │     │
│ │               │   │   },                                                          │     │
│ │               │   │   'W10': {                                                    │     │
│ │               │   │   │   'ip_address': '143.198.29.239',                         │     │
│ │               │   │   │   'sshkey': '/Users/thebes/.ssh/marius'                   │     │
│ │               │   │   },                                                          │     │
│ │               │   │   'W11': {                                                    │     │
│ │               │   │   │   'ip_address': '143.198.29.246',                         │     │
│ │               │   │   │   'sshkey': '/Users/thebes/.ssh/marius'                   │     │
│ │               │   │   },                                                          │     │
│ │               │   │   'W12': {                                                    │     │
│ │               │   │   │   'ip_address': '159.203.77.178',                         │     │
│ │               │   │   │   'sshkey': '/Users/thebes/.ssh/marius'                   │     │
│ │               │   │   },                                                          │     │
│ │               │   │   'W13': {                                                    │     │
│ │               │   │   │   'ip_address': '143.198.19.31',                          │     │
│ │               │   │   │   'sshkey': '/Users/theb